In [13]:
#!/usr/bin/env python3
import gym
from collections import namedtuple
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128   # Kích thước của hidden layer trong mạng nơ-ron
BATCH_SIZE = 16     # Kích thước của mỗi batch (số lượng trạng thái trong mỗi batch)
PERCENTILE = 70     # Phần trăm percentile (dùng để chọn ra các trạng thái tốt nhất)
MAX_EPISODES = 200  # Số lượng episodes tối đa

In [14]:
class Net(nn.Module):
    '''
    Mạng nơ-ron đơn giản với 1 lớp ẩn và 1 lớp đầu ra
    '''
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size), # Lớp đầu vào có obs_size đầu vào và hidden_size đầu ra
            nn.ReLU(),                        # Lớp kích hoạt ReLU 
            nn.Linear(hidden_size, n_actions) # Lớp đầu ra có hidden_size đầu vào và n_actions đầu ra
        )

    def forward(self, x):
        '''
        Hàm forward tính toán đầu ra của mạng nơ-ron
        '''
        return self.net(x)

In [15]:
# Khởi tạo tuples để lưu trữ episode và bước
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

def iterate_batches(env, net, batch_size):
    '''
    Dùng để tạo ra các batch từ môi trường và mạng neural network,
    mỗi batch chứa một số episode và mỗi episode chứa một số bước
    - env: môi trường
    - net: mạng neural network
    - batch_size: số episode trong một batch
    '''

    # Khởi tạo một list để chứa các episode
    batch = []
    # Khởi tạo reward của một episode
    episode_reward = 0.0
    # Khởi tạo list để chứa các bước trong một episode
    episode_steps = []
    # Khởi tạo môi trường
    obs, _ = env.reset()
    # Khởi tạo softmax layer
    sm = nn.Softmax(dim=1)

    # Lặp qua các episode
    while True:
        # Chuyển observation về dạng tensor
        obs_v = torch.FloatTensor([obs])
        # Tính toán output từ mạng neural network
        act_probs_v = sm(net(obs_v))
        # Chuyển output về dạng numpy
        act_probs = act_probs_v.data.numpy()[0]

        # Chọn action dựa trên output một cách ngẫu nhiên
        action = np.random.choice(len(act_probs), p=act_probs)

        # Thực hiện action và nhận lại observation, reward, is_done
        next_obs, reward, is_done, _, _ = env.step(action)

        # Cộng dồn reward của episode
        episode_reward += reward

        # Tạo một bước trong episode và thêm vào list episode_steps
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)

        # Nếu episode kết thúc
        if is_done:
            # Tạo một episode mới và thêm vào list batch
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            # Reset các biến
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()

            # Nếu list batch đủ lớn thì trả về list batch
            if len(batch) == batch_size:
                # Trả về list batch
                yield batch
                batch = []
        
        # Cập nhật observation
        obs = next_obs



In [16]:
def filter_batch(batch, percentile):
    # Tạo một list chứa các reward của các episode trong batch
    rewards = list(map(lambda s: s.reward, batch))
    # Tính reward_bound từ percentile của list trên
    reward_bound = np.percentile(rewards, percentile)
    # Tính reward_mean từ mean của list trên
    reward_mean = float(np.mean(rewards))

    # Tạo 2 list rỗng để chứa các observation và action
    train_obs = []
    train_act = []

    # Duyệt qua từng episode trong batch
    for reward, steps in batch:
        # Nếu reward của episode < reward_bound thì bỏ qua
        if reward < reward_bound:
            continue

        # Nếu không thì thêm observation và action của từng step trong episode vào 2 list trên
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


In [17]:
if __name__ == "__main__":
    # Khởi tạo môi trường CartPole-v0 của OpenAI Gym (tối đa 200 episodes)
    env = gym.make("CartPole-v0", render_mode="human")
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    # Khởi tạo mô hình
    net = Net(obs_size, HIDDEN_SIZE, n_actions)

    # Sử dụng CrossEntropyLoss để tính loss
    objective = nn.CrossEntropyLoss()

    # Sử dụng Adam optimizer để tối ưu hóa mô hình
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)

    # Huấn luyện mô hình và tối ưu hóa
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        # Lọc ra các batch có reward cao
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        
        # Tối ưu hóa mô hình bằng cách tính loss và backward
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()

        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (iter_no, loss_v.item(), reward_m, reward_b))

        # Kiểm tra xem mô hình đã giải quyết bài toán chưa
        if reward_m >= MAX_EPISODES:
            print("Solved!")
            break


0: loss=0.702, reward_mean=19.6, rw_bound=21.5
1: loss=0.680, reward_mean=26.8, rw_bound=34.5
2: loss=0.669, reward_mean=24.2, rw_bound=28.0
3: loss=0.648, reward_mean=34.8, rw_bound=45.0
4: loss=0.628, reward_mean=34.5, rw_bound=41.0
5: loss=0.632, reward_mean=44.1, rw_bound=49.5
6: loss=0.626, reward_mean=40.1, rw_bound=42.5
7: loss=0.602, reward_mean=54.3, rw_bound=69.5
8: loss=0.606, reward_mean=53.2, rw_bound=65.0
9: loss=0.593, reward_mean=62.8, rw_bound=62.0
10: loss=0.588, reward_mean=54.4, rw_bound=62.5
11: loss=0.564, reward_mean=64.2, rw_bound=80.0
12: loss=0.558, reward_mean=78.6, rw_bound=91.5
13: loss=0.545, reward_mean=71.5, rw_bound=78.5
14: loss=0.554, reward_mean=75.0, rw_bound=83.5
15: loss=0.548, reward_mean=69.4, rw_bound=80.5
16: loss=0.529, reward_mean=76.8, rw_bound=88.5
17: loss=0.542, reward_mean=93.8, rw_bound=112.0
18: loss=0.541, reward_mean=90.2, rw_bound=91.5
19: loss=0.523, reward_mean=93.0, rw_bound=117.0
20: loss=0.538, reward_mean=97.8, rw_bound=108.5